## pydiode (redundancy=1) - dd_udp.c - 1 MiB rcv socket buffer

In [9]:
import pandas as pd

tests1 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-1mib/dd_udp.c/pydiode-1/testresults_pydiode_1_100000000.csv')
tests2 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-1mib/dd_udp.c/pydiode-1/testresults_pydiode_1_250000000.csv')
tests3 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-1mib/dd_udp.c/pydiode-1/testresults_pydiode_1_500000000.csv')
tests4 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-1mib/dd_udp.c/pydiode-1/testresults_pydiode_1_750000000.csv')
tests5 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-1mib/dd_udp.c/pydiode-1/testresults_pydiode_1_1000000000.csv')
results = pd.concat([tests1, tests2, tests3, tests4, tests5])
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bitrate            5000 non-null   int64  
 1   trial              5000 non-null   int64  
 2   sender_rc          5000 non-null   int64  
 3   receiver_rc        5000 non-null   int64  
 4   Xfer_time          5000 non-null   float64
 5   Xfer_success       5000 non-null   bool   
 6   in_file_size       5000 non-null   int64  
 7   out_file_size      5000 non-null   int64  
 8   dtgrms_sent        5000 non-null   int64  
 9   dtgrms_received    5000 non-null   int64  
 10  dtgrms_corrupt     5000 non-null   object 
 11  tx_errors          5000 non-null   int64  
 12  tx_dropped         5000 non-null   int64  
 13  rx_errors          5000 non-null   int64  
 14  rx_dropped         5000 non-null   int64  
 15  snmp_OutDatagrams  5000 non-null   int64  
 16  snmp_SndbufErrors  5000 non-nu

#### Transfer success vs Transfer failure

In [10]:
results['Xfer_success'].value_counts()

Xfer_success
True     4995
False       5
Name: count, dtype: int64

#### Did we lose Frames at the Link layer?
In that case, at least one of 'tx_errors', 'tx_dropped', 'rx_errors' or 'rx_dropped' should not be zero.....

In [11]:
results.query('(tx_errors != 0) | (tx_dropped != 0) | (rx_errors != 0) | (rx_dropped != 0)').value_counts()

Series([], Name: count, dtype: int64)

#### Did we lose UDP Datagrams at the Transport layer? (needs dd_udp.c module)

In [12]:
results.query('dtgrms_sent != dtgrms_received').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  533    0          0            2.985381   True          125000000     125000000      13575        13574            na              0          0           0          0           13575              1636               13572             0              0                    1
            534    0          0            1.129935   False         125000000     0              13575        13576            na              0          0           0          0           13575              1435               1                 0              0                    1
Name: count, dtype: int64

#### How many trials with receiver socket buffer errors? (lost Segments)

In [13]:
results.query('snmp_RcvbufErrors != 0').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  745    0          0            1.120360   False         125000000     123157800      13575        13575            na              0          0           0          0           13575              1522               13569             3              3                    1
            915    0          0            1.244904   False         125000000     121315600      13575        13575            na              0          0           0          0           13575              1629               13529             43             43                   1
            926    0          0            1.286665   False         125000000     123157800      13575        13575            na              0          0 

#### Does the number of receiver socket InErrors equal the number of receiver buffer errors?

In [14]:
results.query('snmp_InErrors != snmp_RcvbufErrors').value_counts()

Series([], Name: count, dtype: int64)

#### Do all unsuccessful trials match all trials with receiver buffer errors?

In [15]:
results.query('(snmp_RcvbufErrors != 0) & (Xfer_success == False)').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  745    0          0            1.120360   False         125000000     123157800      13575        13575            na              0          0           0          0           13575              1522               13569             3              3                    1
            915    0          0            1.244904   False         125000000     121315600      13575        13575            na              0          0           0          0           13575              1629               13529             43             43                   1
            926    0          0            1.286665   False         125000000     123157800      13575        13575            na              0          0 

#### And what about snmp_SndbufErrors?

In [16]:
results.query('(snmp_SndbufErrors != 0)').value_counts()
#results.query('(snmp_SndbufErrors != 0) & (Xfer_success == True)').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  999    0          0            1.317035   True          125000000     125000000      13575        13575            na              0          0           0          0           13575              1605               13572             0              0                    1
750000000   237    0          0            1.646317   True          125000000     125000000      13575        13575            na              0          0           0          0           13575              1                  13572             0              0                    1
            477    0          0            1.524009   True          125000000     125000000      13575        13575            na              0          0 